In [1]:
import pickle

import numpy as np

In [8]:
with open('data/3_qubit_stab_sorted.data', 'rb') as reader:
    stab_sorted = pickle.load(reader)

In [9]:
stab_matrix = np.loadtxt('data/3_qubit_matrix_sorted.csv', delimiter=',',
                         dtype=complex)

In [10]:
selection = stab_matrix[:, ::8]

In [11]:
stab_matrix[:, 560]

array([-0.5-0.j ,  0. -0.5j,  0. -0.j ,  0. -0.j ,  0. -0.j ,  0. -0.j ,
        0. -0.5j, -0.5-0.j ])

In [12]:
stab_sorted[560]

{'vector': array([[-0.5-0.j ],
        [ 0. -0.5j],
        [ 0. -0.j ],
        [ 0. -0.j ],
        [ 0. -0.j ],
        [ 0. -0.j ],
        [ 0. -0.5j],
        [-0.5-0.j ]]),
 'latex': '\\frac{1}{2}\\big{(}-|000\\rangle-i|001\\rangle-i|110\\rangle-|111\\rangle\\big{)}',
 'check_matrix': array([[1, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0]]),
 'generator_strs': ['XYZ', 'IZY', 'ZZI']}

In [13]:
stab_sorted[561]

{'vector': array([[-0. -0.j ],
        [-0. -0.j ],
        [ 0.5-0.j ],
        [-0. -0.5j],
        [-0. -0.5j],
        [ 0.5-0.j ],
        [-0. -0.j ],
        [-0. -0.j ]]),
 'latex': '\\frac{1}{2}\\big{(}|010\\rangle-i|011\\rangle-i|100\\rangle+|101\\rangle\\big{)}',
 'check_matrix': array([[1, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0]]),
 'generator_strs': ['XYZ', 'IZY', '-ZZI']}

In [14]:
n = 3
lin_dep_mat = np.loadtxt(f'data/{n}_qubit_nullspace_nice.csv', delimiter=',', dtype=complex)
updated_mat = (lin_dep_mat * np.sqrt(2)).round(3)

In [15]:
updated_mat[updated_mat == 1.414] = np.sqrt(2)

In [31]:
np.savetxt(f'data/{n}_qubit_B.csv', updated_mat, delimiter=',', fmt='%f')

------

In [2]:
n = 5
with open(f'data/{n}_qubit_subgroups.data', 'rb') as reader:
    check_matrices = pickle.load(reader)

In [4]:
def binary_matrix_rank(A):
    """
    Finds the rank of the binary matrix A by effectively converting it
    into row echelon form.

    Parameters
    ----------
    A : np.ndarray

    N.B. This method *changes* the matrix A!

    Returns
    -------
    rank : int
        The rank of matrix A.

    """

    num_rows, num_cols = A.shape
    rank = 0

    for j in range(num_cols):
        # Find the number of rows that have a 1 in the jth column
        rows = []
        for i in range(num_rows):
            if A[i, j] == 1:
                rows.append(i)

        # If the jth column has more than one 1, use row addition to
        # remove all 1s except the first one, then remove the first
        # such row and increase the rank by 1
        if len(rows) >= 1:
            for c in range(1, len(rows)):
                A[rows[c], :] = (A[rows[c], :] + A[rows[0], :]) % 2

            A = np.delete(A, rows[0], 0)
            num_rows -= 1
            rank += 1

    # For each remaining non-zero row in A, increase the rank by 1
    for row in A:
        if sum(row) > 0:
            rank += 1

    return rank

In [13]:
binary_matrix_rank(check_matrices[0][:, :n])

0

In [26]:
check_mats_sorted = sorted(check_matrices, 
                           key=lambda mat: binary_matrix_rank(mat[:, :n]))

In [27]:
check_mats_sorted[:10]

[array([[0, 1]]), array([[1, 0]]), array([[1, 1]])]

In [28]:
with open(f'data/{n}_qubit_subgroups.data', 'wb') as writer:
    pickle.dump(check_mats_sorted, writer)

In [1]:
def num_of_stab_states(n):
    return 2**n * np.prod(2**np.arange(1, n+1) + 1)

In [3]:
num_of_stab_states(5), num_of_stab_states(5) // 2**5

(2423520, 75735)

In [4]:
num_of_stab_states(6), num_of_stab_states(6) // 2**6

(315057600, 4922775)

In [2]:
(315057600 / 2423520 * 170) / 3600

6.138888888888889

------

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as spr

In [10]:
B_2 = spr.load_npz('data/2_qubit_B.npz')
B_2a = spr.load_npz('data/2_qubit_B_a.npz')

In [ ]:
B_2a = B_2a.toarray()
B_2 = B_2.toarray()

In [18]:
np.array_equiv(B_2a, B_2)

True

In [6]:
B_1 = spr.load_npz('data/1_qubit_B.npz')
B_1a = spr.load_npz('data/1_qubit_B_a.npz')

In [7]:
B_1a.toarray()

array([[ 1.        +0.j,  1.        +0.j,  1.        +0.j,
         1.        +0.j],
       [ 1.        +0.j, -1.        +0.j,  0.        +1.j,
         0.        -1.j],
       [-1.41421356+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j, -1.41421356+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j, -1.41421356+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -1.41421356+0.j]])

In [8]:
B_1.toarray()

array([[ 1.        +0.j,  1.        +0.j,  1.        +0.j,
         1.        +0.j],
       [ 1.        +0.j, -1.        +0.j,  0.        +1.j,
         0.        -1.j],
       [-1.41421356+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j, -1.41421356+0.j,  0.        +0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j, -1.41421356+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -1.41421356+0.j]])

In [8]:
nonzero_2 = []
for j in range(B_2.shape[1]):
    nonzero_2.append(B_2[:, j].nonzero()[0])

In [9]:
nonzero_2

[array([0, 1, 4], dtype=int32),
 array([2, 3, 5], dtype=int32),
 array([0, 1, 6], dtype=int32),
 array([2, 3, 7], dtype=int32),
 array([0, 1, 8], dtype=int32),
 array([2, 3, 9], dtype=int32),
 array([ 0,  1, 10], dtype=int32),
 array([ 2,  3, 11], dtype=int32),
 array([ 0,  2, 12], dtype=int32),
 array([ 1,  3, 13], dtype=int32),
 array([ 0,  2, 14], dtype=int32),
 array([ 1,  3, 15], dtype=int32),
 array([ 0,  2, 16], dtype=int32),
 array([ 1,  3, 17], dtype=int32),
 array([ 0,  2, 18], dtype=int32),
 array([ 1,  3, 19], dtype=int32),
 array([ 0,  3, 20], dtype=int32),
 array([ 1,  2, 21], dtype=int32),
 array([ 0,  3, 22], dtype=int32),
 array([ 1,  2, 23], dtype=int32),
 array([ 0,  3, 24], dtype=int32),
 array([ 1,  2, 25], dtype=int32),
 array([ 0,  3, 26], dtype=int32),
 array([ 1,  2, 27], dtype=int32),
 array([ 4,  5, 28], dtype=int32),
 array([ 6,  7, 29], dtype=int32),
 array([ 4,  5, 30], dtype=int32),
 array([ 6,  7, 31], dtype=int32),
 array([ 8,  9, 32], dtype=int32),
 ar

In [4]:
import pickle
B_data = []
with open('data/6_B_data.data', 'rb') as reader:
    try:
        while True:
            B_data.append(pickle.load(reader))
    except EOFError:
        pass

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as spr
import cvx_opti as opt

soln_4 = np.load('data/4_qubit_soln.npy')
soln_4

array([[-1.25000000e-01-5.17766953e-02j],
       [-5.17766953e-02-1.25000000e-01j],
       [-5.17766953e-02-1.25000000e-01j],
       ...,
       [-1.30362392e-14+2.50403518e-15j],
       [-3.88388743e-15-1.55284619e-14j],
       [-2.07205732e-15-2.43098645e-14j]])

In [3]:
test = spr.csc_array(soln_4)

In [11]:
round(test, 3).nonzero()

(array([    0,     1,     2,     3,     4,     5,     6,     7,    16,
           17,    18,    19,    20,    21,    22,    23,   480,   481,
          482,   483,   496,   497,   498,   499,   544,   545,   546,
          547,   560,   561,   562,   563,  4960,  4961,  4976,  4977,
         5024,  5025,  5040,  5041,  5472,  5473,  5488,  5489,  5536,
         5537,  5552,  5553, 20320, 20336, 20384, 20400, 20832, 20848,
        20896, 20912, 28512, 28528, 28576, 28592, 29024, 29040, 29088,
        29104], dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32))

In [13]:
nz_indices = np.nonzero(soln_4.round(3))[0]
nz_indices

array([    0,     1,     2,     3,     4,     5,     6,     7,    16,
          17,    18,    19,    20,    21,    22,    23,   480,   481,
         482,   483,   496,   497,   498,   499,   544,   545,   546,
         547,   560,   561,   562,   563,  4960,  4961,  4976,  4977,
        5024,  5025,  5040,  5041,  5472,  5473,  5488,  5489,  5536,
        5537,  5552,  5553, 20320, 20336, 20384, 20400, 20832, 20848,
       20896, 20912, 28512, 28528, 28576, 28592, 29024, 29040, 29088,
       29104])

In [14]:
nonzeros = soln_4[nz_indices]
print(np.min(nonzeros), np.max(nonzeros))

(-0.12500000000393854-0.05177669531152881j) (0.12500000000865116-0.051776695302483015j)


In [3]:
extent, state_vectors, soln = opt.more_precise_soln(4, soln_4, opt.T_state(4))
extent, soln

(1.883984097463003,
 array([8.47463892e-17+8.57864376e-02j, 6.06601718e-02+6.06601718e-02j,
        6.06601718e-02+6.06601718e-02j, 8.57864376e-02-1.73718641e-16j,
        6.06601718e-02+6.06601718e-02j, 8.57864376e-02-1.98004769e-16j,
        8.57864376e-02+3.71452248e-17j, 6.06601718e-02-6.06601718e-02j,
        6.06601718e-02+6.06601718e-02j, 8.57864376e-02-2.10147834e-16j,
        8.57864376e-02+7.87785883e-17j, 6.06601718e-02-6.06601718e-02j,
        8.57864376e-02+2.32674370e-17j, 6.06601718e-02-6.06601718e-02j,
        6.06601718e-02-6.06601718e-02j, 1.57287182e-16-8.57864376e-02j]))

In [4]:
import mpmath
mpmath.identify(extent)

'((48-sqrt(2048))/2)**2'

In [1]:
import pickle
with open('data/5_B_data.data', 'rb') as f:
    all_of_it = []
    try:
        while True:
            all_of_it.extend(pickle.load(f))
    except EOFError:
        pass

In [2]:
len(all_of_it)

2423488

------

In [1]:
import numpy as np
import scipy.sparse as spr

In [2]:
# Initial: 3277
test = spr.load_npz('B_data/6_qubit_B_0.npz')
# Final: 1927
# 3277 - 1927 = 1350

In [4]:
1350 * 788 / 1024

1038.8671875

In [7]:
# 15683
test = spr.load_npz('data/5_qubit_B.npz')

import sys
print(sys.getsizeof(test))
potentially_better = test.tocsr()
print(type(test))
print(type(potentially_better))
spr.save_npz('data/5_qubit_B_a.npz', potentially_better)

48
<class 'scipy.sparse._csc.csc_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>


In [1]:
import mpmath

In [10]:
mpmath.identify(2.20690)

'(2**(3/56)*3**(75/56)*5**(3/8))/(7**(19/28))'

In [7]:
(2**(319/30)*3**(13/6))/(5**(167/30))

2.206896579998822